<a href="https://colab.research.google.com/github/RobBurnap/Bioinformatics-MICR4203-MICR5203/blob/main/Notebook/non_homogeneous_CTMC_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Non-Homogeneous CTMC on a Bifurcating Tree (ACGT)

This notebook builds a **toy phylogenetic example** where **different branches use different substitution models** (non-homogeneous across the tree).  
We will:

1. Define two HKY-like **rate matrices** $Q^{(1)}$ and $Q^{(2)}$ for A,C,G,T (different base frequencies and transition/transversion bias).
2. Build a small **binary tree** (bifurcating) with **branch-specific** $Q$ and branch lengths $t$.
3. Compute **transition probabilities** $P_b = e^{Q_b t_b}$ **per branch**.
4. Show the **graphical tree**, **Q/P tables**, and compute a **single-site likelihood** via **Felsenstein's pruning** using **branch-specific** $P_b$.
5. Provide clear formulas and code cells you can tweak for an exercise.


In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import expm

states = ["A","C","G","T"]
idx = {s:i for i,s in enumerate(states)}

def hky_Q(pi, kappa):
    pi = np.asarray(pi, dtype=float)
    pi = pi / pi.sum()
    R = np.ones((4,4), dtype=float)
    transitions = [(idx["A"], idx["G"]), (idx["G"], idx["A"]), (idx["C"], idx["T"]), (idx["T"], idx["C"])]
    for i,j in transitions:
        R[i,j] = kappa
    np.fill_diagonal(R, 0.0)
    Q = R * pi[np.newaxis,:]
    np.fill_diagonal(Q, 0.0)
    np.fill_diagonal(Q, -Q.sum(axis=1))
    rate = np.sum(pi * (-np.diag(Q)))
    Q /= rate
    return Q, pi

def df_mat(M, title=None, r=6):
    import pandas as pd
    df = pd.DataFrame(np.round(M, r), index=states, columns=states)
    if title:
        print(title)
    display(df)
    return df

def draw_tree(branches):
    xy = {
        "R": (0.0, 0.5),
        "X": (0.5, 0.75),
        "Y": (0.5, 0.25),
        "A": (1.0, 0.9),
        "B": (1.0, 0.6),
        "C": (1.0, 0.4),
        "D": (1.0, 0.1),
    }
    plt.figure(figsize=(7,3.5))
    for (u,v,t,label) in branches:
        x1,y1 = xy[u]; x2,y2 = xy[v]
        plt.plot([x1,x2],[y1,y2])
        midx, midy = (x1+x2)/2, (y1+y2)/2
        plt.text(midx, midy+0.03, f"t={t}", ha="center", va="bottom")
        plt.text(midx, midy-0.05, f"{label}", ha="center", va="top")
    for n,(x,y) in xy.items():
        plt.scatter([x],[y])
        plt.text(x, y+0.04, n, ha="center")
    plt.axis("off")
    plt.title("Binary tree with branch-specific models")
    plt.show()

def branch_transition(Q, t):
    return expm(Q * t)



## Models and Tree
- **Model 1**: $\pi^{(1)} = [0.30, 0.20, 0.20, 0.30]$, $\kappa^{(1)} = 4.0$  
- **Model 2**: $\pi^{(2)} = [0.25, 0.25, 0.30, 0.20]$, $\kappa^{(2)} = 1.2$

Tree (rooted at R):

- R → X (Model 1, $t=0.4$)  
- R → Y (Model 2, $t=0.6$)  
- X → A (Model 1, $t=0.3$)  
- X → B (Model 2, $t=0.2$)  
- Y → C (Model 1, $t=0.5$)  
- Y → D (Model 2, $t=0.1$)


In [ ]:

Q1, pi1 = hky_Q([0.30, 0.20, 0.20, 0.30], kappa=4.0)
Q2, pi2 = hky_Q([0.25, 0.25, 0.30, 0.20], kappa=1.2)

print("Model 1: pi=", np.round(pi1,4), "kappa=4.0 (E[rate]=1)")
df_mat(Q1, "Q^(1) (rows sum to 0)")

print("\nModel 2: pi=", np.round(pi2,4), "kappa=1.2 (E[rate]=1)")
df_mat(Q2, "Q^(2) (rows sum to 0)")


In [ ]:

branches = [
    ("R","X",0.4,"Model 1"),
    ("R","Y",0.6,"Model 2"),
    ("X","A",0.3,"Model 1"),
    ("X","B",0.2,"Model 2"),
    ("Y","C",0.5,"Model 1"),
    ("Y","D",0.1,"Model 2"),
]
draw_tree(branches)



## Per-Branch Transition Probabilities
For each branch $b$ with model $Q_b$ and length $t_b$,
$$
P_b = e^{Q_b t_b}
$$


In [ ]:

def get_P(label, t):
    return expm((Q1 if label=="Model 1" else Q2) * t)

Ps = {}
for (u,v,t,label) in branches:
    P = get_P(label, t)
    Ps[(u,v)] = P
    df_mat(P, f"P({u}→{v}) for {label} with t={t}")
    print("Row sums:", np.round(P.sum(axis=1), 6), "\n")



## Single-Site Likelihood (Branch-Specific Pruning)
Observed **tip states**:
- A: A, B: G, C: C, D: T

We compute leaf likelihoods, then propagate up the tree using the appropriate
branch transition matrices, and finally average over root states.


In [ ]:

obs = {"A":"A", "B":"G", "C":"C", "D":"T"}

def leaf_L(node):
    vec = np.zeros(4); vec[idx[obs[node]]] = 1.0
    return vec

LA, LB, LC, LD = leaf_L("A"), leaf_L("B"), leaf_L("C"), leaf_L("D")

P_XA, P_XB = Ps[("X","A")], Ps[("X","B")]
P_YC, P_YD = Ps[("Y","C")], Ps[("Y","D")]
P_RX, P_RY = Ps[("R","X")], Ps[("R","Y")]

def push_child(L_child, P_parent_to_child):
    return P_parent_to_child @ L_child

L_X = push_child(LA, P_XA) * push_child(LB, P_XB)
L_Y = push_child(LC, P_YC) * push_child(LD, P_YD)
L_R = (P_RX @ L_X) * (P_RY @ L_Y)

pi_root = pi1  # choose Model 1's equilibrium as the root prior
site_like = float(pi_root @ L_R)

print("L_X(k):", np.round(L_X, 6))
print("L_Y(k):", np.round(L_Y, 6))
print("L_R(k):", np.round(L_R, 6))
print("Root prior pi:", np.round(pi_root, 4))
print("Single-site likelihood:", site_like)



## Exercises
- Change branch lengths and re-run.
- Swap which branches use Model 1 vs. Model 2.
- Change the tip observations or the root prior $\pi$.
- Set all branches to **Model 1** (homogeneous) and compare the likelihood.
